In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score, precision_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV

# Load and prepare data
data = pd.read_csv('Klasifikasi Katarak.csv', header=None, decimal=',')
data.columns = ['Feature1', 'Feature2', 'Feature3', 'Label']

# Prepare features and labels
X = data[['Feature1', 'Feature2', 'Feature3']].values
y = data['Label'].values

# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Create model with optimized parameters
model = MLPClassifier(
    hidden_layer_sizes=(100, 50),        # Simplified architecture
    activation='tanh',                   # Suitable activation for 'lbfgs'
    solver='lbfgs',                      # Using 'lbfgs' solver
    early_stopping=True,                 # Enable early stopping
    validation_fraction=0.2,             # Fraction for validation
    max_iter=50000,                      # Increased iterations
    random_state=42,                     # For reproducibility
    tol=1e-8,                            # Increased tolerance
    verbose=True                         # Enable verbose to monitor training
)

# Split data with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Train model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest Set Accuracy: {accuracy * 100:.2f}%")

# Calculate sensitivity and specificity for each class
recall = recall_score(y_test, y_pred, average=None)
specificity = []
cm = confusion_matrix(y_test, y_pred)
for i in range(len(le.classes_)):
    tn = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])
    fp = cm[:, i].sum() - cm[i, i]
    specificity.append(tn / (tn + fp))
for idx, label in enumerate(le.classes_):
    print(f"{label} - Sensitivity: {recall[idx] * 100:.2f}%, Specificity: {specificity[idx] * 100:.2f}%")
# Print detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Visualize confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_,
            yticklabels=le.classes_)
plt.title(f'Confusion Matrix (Accuracy: {accuracy * 100:.2f}%)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()
